In [1]:
!pip install langchain


In [2]:
!pip install "unstructured[local-inference]"

  Using cached unstructured-0.7.2-py3-none-any.whl (1.3 MB)
  Using cached argilla-1.8.0-py3-none-any.whl (2.4 MB)
  Using cached msg_parser-1.2.0-py2.py3-none-any.whl (101 kB)
  Using cached pdfminer.six-20221105-py3-none-any.whl (5.6 MB)
  Using cached pypandoc-1.11-py3-none-any.whl (20 kB)
  Using cached python-docx-0.8.11.tar.gz (5.6 MB)
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
  Using cached python-pptx-0.6.21.tar.gz (10.1 MB)
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
  Using cached python_magic-0.4.27-py2.py3-none-any.whl (13 kB)
  Using cached tabulate-0.9.0-py3-none-any.whl (35 kB)
  Using cached xlrd-2.0.1-py2.py3-none-any.whl (96 kB)
  Using cached unstructured_inference-0.5.1-py3-none-any.whl (39 kB)
  Using cached layoutparser-0.3.4-py3-none-any.whl (19.2 MB)
  Using cached opencv_python-4.7.0.72-cp37-abi3-win_amd64.whl (38.2 MB)
  Using cached onnxruntim

ERROR: Cannot install layoutparser[layoutmodels,tesseract]==0.1.0 and layoutparser[layoutmodels,tesseract]==0.1.1 because these package versions have conflicting dependencies.
ERROR: ResolutionImpossible: for help visit https://pip.pypa.io/en/latest/topics/dependency-resolution/#dealing-with-dependency-conflicts


In [3]:
import sys
print(sys.path)

['c:\\Users\\jaden\\OneDrive\\Coding\\Python\\ProjectStudy', 'c:\\Users\\jaden\\OneDrive\\Coding\\Program\\Python 3.11\\python311.zip', 'c:\\Users\\jaden\\OneDrive\\Coding\\Program\\Python 3.11\\DLLs', 'c:\\Users\\jaden\\OneDrive\\Coding\\Program\\Python 3.11\\Lib', 'c:\\Users\\jaden\\OneDrive\\Coding\\Program\\Python 3.11', '', 'c:\\Users\\jaden\\OneDrive\\Coding\\Program\\Python 3.11\\Lib\\site-packages', 'c:\\Users\\jaden\\OneDrive\\Coding\\Program\\Python 3.11\\Lib\\site-packages\\win32', 'c:\\Users\\jaden\\OneDrive\\Coding\\Program\\Python 3.11\\Lib\\site-packages\\win32\\lib', 'c:\\Users\\jaden\\OneDrive\\Coding\\Program\\Python 3.11\\Lib\\site-packages\\Pythonwin']


In [4]:
# Youtube_llm

from langchain.document_loaders import YoutubeLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.vectorstores import FAISS
from langchain.llms import OpenAI
from langchain import PromptTemplate
from langchain.chains import LLMChain
from dotenv import find_dotenv, load_dotenv
import textwrap

load_dotenv(find_dotenv())
embeddings = OpenAIEmbeddings()


def create_db_from_youtube_video_url(video_url: str) -> FAISS:
    loader = YoutubeLoader.from_youtube_url(video_url)
    transcript = loader.load()

    text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=100)
    docs = text_splitter.split_documents(transcript)

    db = FAISS.from_documents(docs, embeddings)
    return db


def get_response_from_query(db, query, k=4):
    """
    text-davinci-003 can handle up to 4097 tokens. Setting the chunksize to 1000 and k to 4 maximizes
    the number of tokens to analyze.
    """

    docs = db.similarity_search(query, k=k)
    docs_page_content = " ".join([d.page_content for d in docs])

    llm = OpenAI(model_name="text-davinci-003")

    prompt = PromptTemplate(
        input_variables=["question", "docs"],
        template="""
        You are a helpful assistant that that can answer questions about youtube videos 
        based on the video's transcript.
        
        Answer the following question: {question}
        By searching the following video transcript: {docs}
        
        Only use the factual information from the transcript to answer the question.
        
        If you feel like you don't have enough information to answer the question, say "I don't know".
        
        Your answers should be verbose and detailed.
        """,
    )

    chain = LLMChain(llm=llm, prompt=prompt)

    response = chain.run(question=query, docs=docs_page_content)
    response = response.replace("\n", "")
    return response, docs


# Example usage:
video_url = "https://www.youtube.com/watch?v=L_Guz73e6fw"
db = create_db_from_youtube_video_url(video_url)

query = "What are they saying about Microsoft?"
response, docs = get_response_from_query(db, query)
print(textwrap.fill(response, width=85))

ModuleNotFoundError: No module named 'langchain.document_loaders'

In [4]:
# Youtube_chat
import yaml
from langchain.document_loaders import YoutubeLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.vectorstores import FAISS
from langchain.chat_models import ChatOpenAI
from langchain.chains import LLMChain
from dotenv import find_dotenv, load_dotenv
from langchain.prompts.chat import (
    ChatPromptTemplate,
    SystemMessagePromptTemplate,
    HumanMessagePromptTemplate,
)
import textwrap

load_dotenv(find_dotenv())
embeddings = OpenAIEmbeddings()


def create_db_from_youtube_video_url(video_url):
    loader = YoutubeLoader.from_youtube_url(video_url)
    transcript = loader.load()

    text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=100)
    docs = text_splitter.split_documents(transcript)

    db = FAISS.from_documents(docs, embeddings)
    return db


def get_response_from_query(db, query, k=4):
    """
    gpt-3.5-turbo can handle up to 4097 tokens. Setting the chunksize to 1000 and k to 4 maximizes
    the number of tokens to analyze.
    """

    docs = db.similarity_search(query, k=k)
    docs_page_content = " ".join([d.page_content for d in docs])

    chat = ChatOpenAI(model_name="gpt-3.5-turbo", temperature=0.2)

    # Template to use for the system message prompt
    template = """
        You are a helpful assistant that that can answer questions about youtube videos 
        based on the video's transcript: {docs}
        
        Only use the factual information from the transcript to answer the question.
        
        If you feel like you don't have enough information to answer the question, say "I don't know".
        
        Your answers should be verbose and detailed.
        """

    system_message_prompt = SystemMessagePromptTemplate.from_template(template)

    # Human question prompt
    human_template = "Answer the following question: {question}"
    human_message_prompt = HumanMessagePromptTemplate.from_template(human_template)

    chat_prompt = ChatPromptTemplate.from_messages(
        [system_message_prompt, human_message_prompt]
    )

    chain = LLMChain(llm=chat, prompt=chat_prompt)

    response = chain.run(question=query, docs=docs_page_content)
    response = response.replace("\n", "")
    return response, docs


# Example usage:
video_url = "https://www.youtube.com/watch?v=L_Guz73e6fw"
db = create_db_from_youtube_video_url(video_url)

query = "What are they saying about Microsoft?"
response, docs = get_response_from_query(db, query)
print(textwrap.fill(response, width=50))

ModuleNotFoundError: No module named 'langchain.document_loaders'